In [1]:
import pandas as pd
import numpy as np

In [43]:
# Read input data from FRBNY website
mex_gdp =  pd.read_excel("inputData/mex_data.xlsx", sheet_name="GDP")
mex_gdp = mex_gdp.set_index("Date")
mex_gdp['gdp.log'] = np.log(mex_gdp['gdp.raw'])
#drop gdp raw 
mex_gdp = mex_gdp.drop(columns = ['gdp.raw'])
mex_gdp






,gdp.log
Date,
1993-01-01,16.124658
1993-04-01,16.127315
1993-07-01,16.137493
1993-10-01,16.150647
1994-01-01,16.160783
...,...
2022-01-01,16.711318
2022-04-01,16.722300
2022-07-01,16.731408


In [44]:
mex_inflation =  pd.read_excel("inputData/mex_data.xlsx", sheet_name="Inflation")
mex_inflation = mex_inflation.set_index("Date")
mex_inflation['index'] = np.nan
#create index from monthly increase 
#set first row to 100
mex_inflation['index'].iloc[0] = 100
mex_inflation
for i in range(1,len(mex_inflation)):
    mex_inflation['index'][i] = ((mex_inflation['inflation.mensual'][i]/100) * mex_inflation['index'][i-1]) + mex_inflation['index'][i-1]

#resample 'index'column to quarterly
mex_inflation = mex_inflation['index'].resample('QS').mean()
mex_inflation['growth'] = mex_inflation.pct_change() * 100
mex_inflation['growth'] = mex_inflation['growth'].dropna()

#annualize quarterly growth
mex_inflation['annualized'] =  100 * ((1 + mex_inflation['growth']/100)**4 - 1)
mex_inflation['annualized']
mex_inflation_final = mex_inflation['annualized'].to_frame()
mex_inflation_final['expectations'] = mex_inflation_final.rolling(4).mean()
mex_inflation_final


,index,expectations
Date,,
1982-04-01,71.596407,NaN
1982-07-01,107.617491,NaN
1982-10-01,109.303424,NaN
1983-01-01,155.936492,111.113454
1983-04-01,81.410336,113.566936
...,...,...
2024-01-01,5.131156,4.639530
2024-04-01,3.463854,4.223897
2024-07-01,3.096590,3.982986


In [45]:
#.interest    
mex_interest =  pd.read_excel("inputData/mex_data.xlsx", sheet_name="Interest")
mex_interest = mex_interest.set_index("Date")
mex_interest = mex_interest.resample('QS').mean()
mex_interest['interest'] = mex_interest['interest'].dropna()
mex_interest

,interest
Date,
2008-01-01,7.500000
2008-04-01,7.527344
2008-07-01,8.073077
2008-10-01,8.250000
2009-01-01,7.622222
...,...
2024-01-01,11.222527
2024-04-01,11.000000
2024-07-01,10.846154


In [46]:
#merge mex_gdp, mex_inflation_final, mex_interest
mex_data = pd.concat([mex_gdp, mex_inflation_final, mex_interest], axis=1).dropna()
mex_data

,gdp.log,index,expectations,interest
Date,,,,
2008-01-01,16.519310,4.950159,3.941631,7.500000
2008-04-01,16.526649,5.479732,4.561684,7.527344
2008-07-01,16.529959,5.493759,5.019913,8.073077
2008-10-01,16.513496,5.367088,5.322684,8.250000
2009-01-01,16.461015,5.998427,5.584751,7.622222
...,...,...,...,...
2022-01-01,16.711318,8.402055,6.550626,5.811111
2022-04-01,16.722300,9.003384,7.361280,6.830556
2022-07-01,16.731408,8.674146,8.012917,8.162088


In [47]:
#rename columns
mex_data.columns = ['gdp.log', 'inflation', 'inflation.expectations', 'interest']

#add columns of 0 called covid.indicator
mex_data['covid.indicator'] = 0

#save data to excel
mex_data.to_excel("inputData/Holston_Laubach_Williams_MEX.xlsx", sheet_name="MEX Input Data" )
